In [1]:
import cv2
import numpy as np

# load pre-trained models
print("loading models.....")
net = cv2.dnn.readNetFromCaffe('colorization_deploy_v2.prototxt', 'colorization_release_v2.caffemodel')
pts = np.load('pts_in_hull.npy')

# configure network for colorization
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype='float32')]



loading models.....


In [ ]:
# read black and white input video
input_video_path = "waves2.mp4"
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# create output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = "waves2_colorized.mp4"
out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

# process frames and write to output video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # prepare black and white frame for colorization
    scaled = frame.astype("float32") / 255.0
    lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
    resized = cv2.resize(lab, (224, 224))
    L = cv2.split(resized)[0]
    L -= 50
    net.setInput(cv2.dnn.blobFromImage(L))
    
    # colorize frame
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    ab = cv2.resize(ab, (frame.shape[1], frame.shape[0]))
    L = cv2.split(lab)[0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)
    colorized = (255 * colorized).astype("uint8")
    
    # write colorized frame to output video
    out.write(colorized)

cap.release()
out.release()
cv2.destroyAllWindows()
